In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#Loading Train and Test Data
train = pd.read_csv('/home/ashwani/Desktop/Python/DataSet/loan/train.csv')
test = pd.read_csv('/home/ashwani/Desktop/Python/DataSet/loan/test.csv')


In [ ]:
#shape of train and test data
print(train.shape)
print(test.shape)


In [ ]:
# Fist 6 row of train dataset
train.head(6)

In [ ]:
# Variables of train dataset
train.columns

In [ ]:
#Get summary of numerical variable 

train.describe()

In [ ]:
# summary of categorical variables
train.describe(include=['O'])

In [ ]:
train['Property_Area'].value_counts()

In [ ]:
train['Dependents'].value_counts()

In [ ]:
#getting missing value
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#Replace missing value of train test data set
train['LoanAmount'].fillna(train['LoanAmount'].mean(), inplace=True)
test['LoanAmount'].fillna(test['LoanAmount'].mean(), inplace=True)


In [ ]:
train['Self_Employed'].value_counts()
train['Self_Employed'].fillna('No',inplace=True)
test['Self_Employed'].fillna('No',inplace=True)

In [ ]:
train['Gender'].value_counts()
train['Gender'].fillna('Male', inplace=True)
test['Gender'].fillna('Male', inplace=True)

In [ ]:
train['Credit_History'].value_counts()
train['Credit_History'].fillna('1.0', inplace=True)
test['Credit_History'].fillna('1.0', inplace=True)

In [ ]:
train['Loan_Amount_Term'].value_counts()
train["Loan_Amount_Term"].fillna('360.0', inplace=True)
test["Loan_Amount_Term"].fillna('360.0', inplace=True)


In [ ]:
train['Dependents'].value_counts()
train['Dependents'].fillna('0', inplace=True)
test['Dependents'].fillna('0', inplace=True)

In [ ]:
train['Married'].value_counts()
train['Married'].fillna('Yes', inplace=True)

In [ ]:
#create a feature variable Credibility
train['Credibility'] = train['ApplicantIncome'] / train['LoanAmount']
test['Credibility'] = test['ApplicantIncome'] / test['LoanAmount']

In [ ]:
cat= ['Gender','Married','Dependents','Self_Employed','Property_Area','Education']
num = ['ApplicantIncome', 'CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History']

In [ ]:
for col  in num:
    train[col]=train[col].astype('float64')
    test[col]=test[col].astype('float64')

In [ ]:
for col  in cat:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')

In [ ]:
final=train.append(test, ignore_index=True)
final=final.drop(['Loan_ID'], axis=1)
final.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in ['Dependents']:
    final[col] = le.fit_transform(final[col])
    
final_wt=pd.get_dummies(final.drop(['Loan_Status'], axis=1))

In [ ]:
final_wt['Loan_Status']=final['Loan_Status']
final_wt.dtypes

In [ ]:
final_wt['Loan_Status']=final_wt['Loan_Status'].astype('category')


In [ ]:
train=final_wt.iloc[0:614,:]
test=final_wt.iloc[614:,:]

In [ ]:
test_loan=test.drop(['Loan_Status'], axis=1)

In [ ]:
train.shape, test_loan.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(train.drop(['Loan_Status'], axis=1),
                                                  train['Loan_Status'],test_size =0.4,
                                                    random_state = 123)

In [ ]:
# Decesion Tree
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier


tree_model = DecisionTreeClassifier()
tree_model.fit(x_train, y_train)
tree_pred= tree_model.predict(x_test)

metrics.accuracy_score(y_test,tree_pred), metrics.cohen_kappa_score(y_test,tree_pred)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rand_model = RandomForestClassifier(n_estimators=700)
rand_model.fit(x_train, y_train)
rand_pred= rand_model.predict(x_test)
metrics.confusion_matrix(y_test,rand_pred)
metrics.accuracy_score(y_test,rand_pred), metrics.cohen_kappa_score(y_test,rand_pred)

In [ ]:
from sklearn import svm
from sklearn.preprocessing import scale
X_train=scale(x_train, axis=0)
X_test=scale(x_test, axis=0)
svm_model = svm.SVC()
svm_model.fit(X = X_train, y = y_train)
pred_svm = svm_model.predict(X_test)
metrics.confusion_matrix(y_test,pred_svm)
metrics.accuracy_score(y_test,pred_svm), metrics.cohen_kappa_score(y_test,pred_svm)

In [ ]:
pred_test=rand_model.predict(test_loan)

In [ ]:
test_output=pd.DataFrame({'Loan_ID':test_loan['Loan_ID'], 'Loan_Status':pred_test})

test_output.to_csv('/home/ashwani/Desktop/loan_output.csv',index=False)